# Finite State Machines 

Overview

1. Fundamentals
2. Turnstile
3. Traffic Jam
4. Sudoku

But first we must be ready:

In [2]:
import sys
assert (sys.version_info.major, sys.version_info.minor) >= (3,7)
%pip install graph-theory --upgrade --no-cache -q
from graph import Graph

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Fundamentals

Nomenclature

- A finite state machine is a mathematical model of computation. 
- The model is in exactly one state at any time.
- The model can change from one state to another using predetermined paths.
- All possible states is called the "solution landscape"



# Turnstile

 ![turnstile machine](images/Torniqueterevolution.jpg) ![turnstile](images/turnstile.png)

In [3]:
from graph.finite_state_machine import FiniteStateMachine

locked, unlocked = 'locked', 'unlocked'  # states
push, coin = 'push', 'coin'  # actions

fsm = FiniteStateMachine()
fsm.add_transition(locked, coin, unlocked)  # turnstile is locked. Put in coin to unlock.
fsm.add_transition(unlocked, push, locked)  # turnstile is unlocked. Push the rotor to lock.
fsm.add_transition(locked, push, locked)  # turnstile is locked. Pushing does not unlock.
fsm.add_transition(unlocked, coin, unlocked)  # turnstile is unlocked. Adding more coins does not change state.

fsm.set_initial_state(locked)  # set initial state.

In [4]:
# check options: pay and go
set(fsm.options())

{'coin', 'push'}

In [5]:
fsm.current_state == locked

True

In [6]:
# now insert coin:
fsm.next(action=coin)

In [7]:
# check options:
set(fsm.options())

{'coin', 'push'}

In [8]:
fsm.current_state == unlocked

True

In [9]:
# ok it's unlocked - so we push:
fsm.next(action=push)
# and check if it locks behind us:
fsm.current_state == locked

True

# Traffic Jam

As finite state machines can be view as a model of *a valid state*, we can use to verify solutions to optimization problems.

In the example below we have a small intersection like, where all the `red`s need to pass all the `blue`s. 

The order could matter, but we ignore that for now.

![3-3 traffic jam](images/3-3-cart-traffic-jam_initial.png)

(initial state)

The solution we would like to see looks like this:

![3-3 traffic jan final](images/3-3-cart-traffic-jam.png)

(final state)

And here's the solution that swaps all the blue and red tiles in fewest moves:

![3-3 traffic jam solution](images/traffic_bi_directional.gif)

Q: How did I get to that?

A: 3 steps:

1. Define the system as a FSM using a "map" of options.
2. Set the `initial state`
3. Search through the solution landscape until the `active state` resembles the `final state`.


We will now solve the same problem using a just a tiny subset.

Let's start with the map:

![tjs-map](images/tjs-map.png)

In [10]:
g = Graph()
edges = [(1,2),(2,3),(2,4)]
for edge in edges:
    g.add_edge(*edge, bidirectional=True)

Next I add the tiles to represent the initial state and the final state:

![tjs with pug](images/tjs-map-loads.png)

In [11]:
loads = [
    {'id':'red', 'start': 1, 'ends': 3}, 
    {'id': 'blue', 'start': 3, 'ends': 1}
]

In [12]:
from graph.traffic_scheduling_problem import jam_solver

In [13]:
solution = jam_solver(g,loads)

queue exhausted


In [14]:
solution

[{'red': (1, 2)},
 {'red': (2, 4), 'blue': (3, 2)},
 {'blue': (2, 1), 'red': (4, 2)},
 {'red': (2, 3)}]

In [15]:
# in plain english:
for move in solution:
    for color,(start,end) in move.items():
        print(f"{color} moves from {start} to {end}")

red moves from 1 to 2
red moves from 2 to 4
blue moves from 3 to 2
blue moves from 2 to 1
red moves from 4 to 2
red moves from 2 to 3


So what kind of `finite state machine` is the traffic jam solver using?

Each state is captured on a graph which represents a `tree of options`. As there are a finite number of options to for changing the finite state machine from it's current state to the next state, we can capture the change as branches our `tree of options`. Every time our "search" generates a novel branch, we add that to the tree. The unexplored options then become our "frontier" from which we search further.

![tjs](images/3-3-tree-of-states.png)

We can make a deep-copy of our state machine, as the tree above illustrates, but will quickly consume a terrible amount of memory, 

To capture each `state` of the `finite state machine` we only need to look at the position of loads. Here is an example:

In [16]:
state_0 = ((1, 'red'), (2, None), (3, 'blue'), (4, None))
state_1 = ((1, None), (2, 'red'), (3, 'blue'), (4, None))
state_2 = ((1, 'red'), (2, 'blue'), (3, None), (4, None))


We can now put these states into our `tree of options` as a graph:

In [17]:

fsm_graph = Graph()
fsm_graph.add_edge(state_0, state_1)  # there is a path from state_0 to state_1
fsm_graph.add_edge(state_0, state_2)  # there is a path from state_0 to state_2

The search now becomes:

1. "make a change", 
2. check if it exists in the graph (and add it to the graph if it is novel). 

However the state definition contains a lot of `None`s that don't really add much.

Suggestion: Let's just drop all the `None`s that information.

New graph:

In [18]:
state_0 = ((1, 'red'), (3, 'blue'))
state_1 = ((2, 'red'), (3, 'blue'))
state_2 = ((1, 'red'), (2, 'blue'))

fsm_graph = Graph()
fsm_graph.add_edge(state_0, state_1)
fsm_graph.add_edge(state_0, state_2)

That's nicer to read.

We can now make a solver that identifies a path from the `initial state` to the `final state`, simply by searching along the frontier from the `initial state`:

In [19]:
# we need the "map" from earlier:
the_map = Graph()
edges = [(1,2),(2,3),(2,4)]
for edge in edges:
    the_map.add_edge(*edge, bidirectional=True)

# we need the "frontier"
job_queue = [state_0, state_1, state_2]  # (you really only need state_0)

# we need a "final state"
final_state = tuple(sorted([(3,'red'), (1,'blue')]))

# we need a search
while job_queue:
    state = job_queue.pop()
    occupied = {position for position,item in state}
    for position, item in state:  # 'red' on position 2.
        for option in the_map.nodes(from_node=position):  # positions {1,4}
            if option in occupied:
                continue  # skip it.
            
            new_state = tuple(sorted([(option, item)] + [(o,i) for o,i in state if i != item]))
            
            if new_state in fsm_graph:
                continue  # we've already seen it.
            
            # we add the new state to be explored.
            fsm_graph.add_edge(state, new_state)
            job_queue.append(new_state)

            if new_state == final_state:
                break

As the solver has made the tree of options of the finite state diagram we can have a quick look:

In [20]:
fsm_graph.nodes()

[((1, 'red'), (3, 'blue')),
 ((2, 'red'), (3, 'blue')),
 ((1, 'red'), (2, 'blue')),
 ((1, 'red'), (4, 'blue')),
 ((2, 'red'), (4, 'blue')),
 ((3, 'red'), (4, 'blue')),
 ((2, 'blue'), (3, 'red')),
 ((1, 'blue'), (3, 'red')),
 ((1, 'blue'), (2, 'red')),
 ((1, 'blue'), (4, 'red')),
 ((2, 'blue'), (4, 'red')),
 ((3, 'blue'), (4, 'red'))]

To find the fewest moves to resolve the traffic jam, we can just ask the finite state diagram for the shortest path from the `initial state` to the `final state`:

In [21]:
fsm_graph.shortest_path(state_0, final_state)

(6,
 [((1, 'red'), (3, 'blue')),
  ((1, 'red'), (2, 'blue')),
  ((1, 'red'), (4, 'blue')),
  ((2, 'red'), (4, 'blue')),
  ((3, 'red'), (4, 'blue')),
  ((2, 'blue'), (3, 'red')),
  ((1, 'blue'), (3, 'red'))])

And there you have it! The solution from earlier:

```
red moves from 1 to 2
red moves from 2 to 4
blue moves from 3 to 2
blue moves from 2 to 1
red moves from 4 to 2
red moves from 2 to 3
```

and the solution above shows, state by state what has changed. 

Note that the two valid solutions interchangeable. 

# Sudoku

Solving a sudoku is no different can be done in a similar way, but our search can become a lot more efficient if we introduce the ideas of a wave collapse function.

Q: A what-collapse?

A: A wave collapse function. 

Q: ???

A: Imagine you're surfing on a wave from an `initial state` over undefined state (US) space. As the wave rolls over the undefined space (US), it imposes the most probably outcome onto the undefined space. Each insertion causes the options of the neighbouring spaces to collapse from all possible options to very few plausible options. [More about wave collapse functions](https://github.com/mxgmn/WaveFunctionCollapse)

![sudoku](images/sudoku-wave-collapse-1.png)

Inserting for example `2` in the red box would immediately make `2` unavailable in all other boxes in top right corner. That is an example of a collapse of options.

We can describe the "options" for each cell as: numbers - (box + row + column), e.g. the  `red` cell as:

In [22]:
{1,2,3,4,5,6,7,8,9} - ({1,3,4,5,6,8} | {3,5} | {3,4,6}) 

{2, 7, 9}

We can also describe a whole sudoku's state as a list numbers, with zero as the uncollapsed space:.

The box above hence becomes: 
```
[8,4,1,
 0,0,6,
 5,0,3]
```

Doing the same for a whole sudoku like this: 

![?](images/easy_sudoku.png)

can then be:



In [23]:
sudoku = [
    # cell value        # cell number
    0,0,4,0,5,0,0,0,0,  #  0, 1, 2, 3, 4, 5, 6, 7, 8,
    9,0,0,7,3,4,6,0,0,  #  9,10,11,12,13,14,15,16,17,
    0,0,3,0,2,1,0,4,9,  # 18,19,20,21,22,23,24,25,26,  
    0,3,5,0,9,0,4,8,0,  # 27,28,29,30,31,32,33,34,35,
    0,9,0,0,0,0,0,3,0,  # 36,37,38,39,40,41,42,43,44,
    0,7,6,0,1,0,9,2,0,  # 45,46,47,48,49,50,51,52,53,
    3,1,0,9,7,0,2,0,0,  # 54,55,56,57,58,59,60,61,62,
    0,0,9,1,8,2,0,0,3,  # 63,64,65,66,67,68,69,70,71,
    0,0,0,0,6,0,1,0,0,  # 72,73,74,75,76,77,78,79,80
    # zero = blank.
]

To describe the options for a particular cell, we now need some basic math:

We find the set of number that have not been used - just like a minute ago - but we use the "cell" based address system.

![field_53 options](images/easy_sudoku3.png)

In [24]:
from math import floor
numbers = {1,2,3,4,5,6,7,8,9}

def options(cell,sudoku):    
    column = {v for ix, v in enumerate(sudoku) if ix % 9 == cell % 9}
    row = {v for ix, v in enumerate(sudoku) if ix // 9 == cell // 9}
    box = {v for ix, v in enumerate(sudoku) if (ix // (9 * 3) == cell // (9 * 3)) and ((ix % 9) // 3 == (cell % 9) // 3)}
    return numbers - (box | row | column)

In [25]:
options(53, sudoku)

{5}

So there's one option for cell number 53, meaning that the solution landscape *must* eventually collapse around this single option. We can now guide our "search" by picking the cell's that have least options.

Let's find them:

In [26]:
degrees_of_freedom = [0 if v!=0 else len(options(ix,sudoku)) for ix,v in enumerate(sudoku)]

for i in range(9):
    print(degrees_of_freedom[i*9:i*9+9])

[5, 3, 0, 2, 0, 3, 3, 2, 4]
[0, 3, 3, 0, 0, 0, 0, 2, 4]
[4, 3, 0, 2, 0, 0, 3, 0, 0]
[2, 0, 0, 2, 0, 2, 0, 0, 3]
[4, 0, 3, 5, 1, 4, 2, 0, 4]
[2, 0, 0, 4, 0, 3, 0, 0, 1]
[0, 0, 1, 0, 0, 1, 0, 2, 4]
[4, 3, 0, 0, 0, 0, 2, 3, 0]
[5, 4, 3, 3, 0, 2, 0, 3, 4]


Each of the numbers are the number of options available for each cell.

We can then design our solver to inspect the sudoku as a finite state machine, where the "current state" is our partial solution, e.g. a list with 81 integers.

![](images/sudoku_solver1.png)

In [27]:
# the sudoku is our initial state.
initial_state = sudoku[:]

job_queue = [initial_state]  # we need the jobqueue in case of ambiguity of choice.

while job_queue:
    state = job_queue.pop(0)
    if not any(i==0 for i in state):  # no missing values means that the sudoku is solved.
        break

    degrees_of_freedom = [0 if v!=0 else len(options(ix,state)) for ix,v in enumerate(state)]
    least_freedom = min(v for v in degrees_of_freedom if v > 0)
    cell = degrees_of_freedom.index(least_freedom)

    for option in options(cell, state):  # for each option we add the new state to the queue.
        new_state = state[:]
        new_state[cell] = option
        job_queue.append(new_state)

# we print out the solution
for i in range(9):
    print(state[i*9:i*9+9])

[2, 6, 4, 8, 5, 9, 3, 1, 7]
[9, 8, 1, 7, 3, 4, 6, 5, 2]
[7, 5, 3, 6, 2, 1, 8, 4, 9]
[1, 3, 5, 2, 9, 7, 4, 8, 6]
[8, 9, 2, 5, 4, 6, 7, 3, 1]
[4, 7, 6, 3, 1, 8, 9, 2, 5]
[3, 1, 8, 9, 7, 5, 2, 6, 4]
[6, 4, 9, 1, 8, 2, 5, 7, 3]
[5, 2, 7, 4, 6, 3, 1, 9, 8]


# Conclusions

What did we learn?

Finite State Machines ...

- are mathematical models of states and transitions.
- can be used as generators for mapping of a solution landscape.
- can be used to solve any discrete problem.